In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [ ]:
from transformers import AutoModelForCausalLM
from utils.torch_utils import count_parameters
import torch

model_path="/home/golympie/ai-toolbox/pruned_models/"+"deepseek_coder_v2_lite_instruct_fused_2"

model=AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map='cpu',
    trust_remote_code=True
)
count_parameters(model)

# Deepseek-Coder-v2-Lite-Instruct_Fused_2B

In [ ]:
help(model.quantize)

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

model_path = "/home/golympie/ai-toolbox/pruned_models/Deepseek-Coder-v2-Lite-Instruct_Fused_2B"
quant_path = "/home/golympie/ai-toolbox/pruned_models/Deepseek-Coder-v2-Lite-Instruct_Fused_2B-AWQ"

quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM",
    # "max_calib_seq_len": 256,
    # "n_parallel_calib_samples":4
}



# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda:1")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(
    tokenizer, 
    quant_config=quant_config,
    max_calib_seq_len=256,
    n_parallel_calib_samples=4
)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

In [ ]:
export OPENAI_API_BASE=http://localhost:5000/v1/models
export OPENAI_API_KEY=c01a4e91d28971ae1a6957d52b55b226

lighteval endpoint litellm openai/gpt-4 mmlu \
--max-samples 64 \
--override-batch-size 16


lighteval|gpqa
lighteval|gsm8k

In [ ]:
from deepeval.models.base_model import DeepEvalBaseLLM


class MirascopeDeepeval(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return self.model

In [ ]:
from deepeval.benchmarks import MMLU
from deepeval.benchmarks.mmlu.task import MMLUTask

benchmark = MMLU(tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY], n_shots=3)

In [ ]:
help(benchmark.evaluate)

In [ ]:
from langchain_openai import AzureChatOpenAI

In [ ]:
import deepeval

In [ ]:
dir(deepeval.models.DeepEvalBaseLLM)